In [1]:
# Library Imports
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


%autosave 60
%matplotlib inline

Autosaving every 60 seconds


In [3]:
from google.colab import files

uploaded = files.upload()

Saving TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv to TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [4]:
ted = pd.read_csv('data/TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv')

In [14]:
ted.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 349 to 2448
Data columns (total 123 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4 non-null      int64  
 1   speaker               4 non-null      object 
 2   headline              4 non-null      object 
 3   URL                   4 non-null      object 
 4   description           4 non-null      object 
 5   transcript_URL        4 non-null      object 
 6   month_filmed          4 non-null      int64  
 7   year_filmed           4 non-null      int64  
 8   event                 4 non-null      object 
 9   duration              4 non-null      object 
 10  date_published        4 non-null      object 
 11  views_as_of_06162017  4 non-null      float64
 12  tags                  4 non-null      object 
 13  transcript            4 non-null      object 
 14  notes                 4 non-null      object 
 15  WC                   

In [6]:
ted.head()
# ted.describe()
# ted

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,notes,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,WC2,WPS.1,Sixltr.1,Dic.1,harm_virtue,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general
0,1,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2,2006,TED2006,0:16:17,6/27/06,3177001.0,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",NaN,2281.0,66.79,75.31,43.31,71.09,15.62,15.65,81.59,51.56,15.61,8.24,3.24,1.62,2.28,0.44,0.66,7.32,6.93,13.42,8.59,4.12,5.61,0.83,16.26,4.12,2.02,1.49,6.97,3.11,4.60,3.46,1.05,0.13,0.18,0.35,9.47,0.26,0.18,0.31,0.35,9.78,2.10,1.84,1.14,1.93,1.27,2.46,3.29,0.75,2.06,0.39,0.75,0.09,0.18,0.00,0.44,6.93,2.50,1.18,2.02,1.32,0.48,4.08,11.14,1.45,13.77,2.24,7.19,4.56,2.37,0.83,0.44,1.40,0.00,0.04,0.22,0.00,0.04,0.09,0.09,0.00,22.80,6.05,5.57,3.20,0.18,0.35,0.26,0.96,0.83,2.76,2.63,0.00,2287.0,15.66,15.52,1.01,0.04,0.04,0.00,0.00,0.48,0.00,0.22,0.0,0.00,0.00,0.22
1,2,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2,2006,TED2006,0:15:06,8/15/06,1379328.0,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",NaN,2687.0,45.13,78.94,19.05,54.50,16.79,14.92,84.85,57.39,17.71,8.22,1.41,2.79,2.16,0.33,1.53,9.49,7.07,13.14,9.68,6.62,7.15,1.27,17.19,3.68,2.23,2.05,4.88,1.49,3.39,2.46,0.93,0.04,0.00,0.30,8.86,0.11,0.11,0.15,0.37,11.46,2.12,2.83,1.94,1.79,0.89,3.09,1.94,0.86,0.41,0.56,2.08,0.07,0.63,0.07,1.38,8.45,3.16,1.82,2.16,1.41,0.41,3.76,12.84,0.63,11.39,1.86,7.03,3.05,3.13,0.78,0.22,1.23,0.00,0.15,0.30,0.00,0.00,0.11,0.15,0.04,16.67,5.62,4.47,1.64,0.07,0.45,0.00,0.74,0.30,2.68,0.67,0.04,2699.0,16.87,14.82,0.82,0.07,0.04,0.00,0.00,0.30,0.00,0.11,0.0,0.11,0.04,0.15
2,3,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7,2005,TEDGlobal 2005,0:18:45,10/18/06,790536.0,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",NaN,2506.0,74.47,65.67,27.20,35.53,15.57,21.55,83.60,53.19,12.93,6.58,2.19,2.00,1.40,0.24,0.76,6.34,7.70,14.88,9.70,3.79,5.95,1.52,15.56,3.75,2.39,2.08,5.03,2.27,4.03,2.23,1.68,0.44,0.32,0.12,8.10,0.00,0.12,0.40,0.08,12.37,2.47,1.84,2.47,2.51,1.32,3.71,1.12,0.44,0.44,0.12,1.04,0.20,0.72,0.00,0.12,7.26,2.51,1.68,2.11,0.88,0.56,2.63,11.57,1.08,11.69,1.48,6.90,3.63,4.23,0.28,0.08,2.15,0.12,0.00,0.44,0.04,0.08,0.24,0.08,0.00,17.24,6.07,6.86,1.28,0.24,0.60,0.04,0.28,0.00,1.72,0.16,0.00,2511.0,15.60,21.43,1.23,0.12,0.12,0.16,0.04,0.32,0.12,0.20,0.0,0.04,0.04,0.08
3,4,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2,2006,TED2006,0:19:37,10/25/06,1985119.0,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",NaN,3092.0,46.11,71.00,70.27,61.34,16.10,15.27,87.9

In [7]:
# get top n most frequent names
ted['speaker'].value_counts()[:10].sort_values(ascending=False) #.index.tolist()

Hans Rosling            9
Juan Enriquez           7
Marco Tempest           6
Rives                   6
Clay Shirky             5
Bill Gates              5
Dan Ariely              5
Jacqueline Novogratz    5
Nicholas Negroponte     5
Julian Treasure         5
Name: speaker, dtype: int64

In [16]:
# ted.transcript[0]

In [9]:
ted.transcript.isnull().sum()

89

In [10]:
ted = ted.dropna()